In [ ]:
import os
import time
from datetime import date, timedelta

import pandas as pd
import numpy as np

import requests 
from bs4 import BeautifulSoup

import io
import re
import itertools
import zipfile

import numpy as np

import pyshp 
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns

import mapclassify
import geopandas as gp
import numpy as np
import sys

from matplotlib import cm
import geopandas

In [ ]:
pays = pd.read_excel("PaysdAfrique.xlsx")

In [ ]:
pays.head()

In [ ]:
evolution = pd.read_excel("regionsAfrique.xlsx")

In [ ]:
evolution.head()

In [ ]:
annee = ["2002","2008","2012","2018"]

In [ ]:
index = ["North", "West", "Central", "East", "South"]
evolution.index = index
evolution

In [ ]:
#Incidence Bladder cancer
bdm = pd.DataFrame(evolution, columns=['Bladder 2002 M', 'Bladder 2012 M', 'Bladder 2018 M']).transpose()
bdw = pd.DataFrame(evolution, columns=['Bladder 2002 F', 'Bladder 2012 F', 'Bladder 2018 F']).transpose()

#Both men and women
bdm.index = ["2002", "2008", "2018"]
bdw.index = ["2002", "2008", "2018"]
Bladdersomme = bdm.add(bdw).transpose()/2
Bladdersomme
#tym, tyw#, Thyroidsomme

In [ ]:
Bladdersomme

middle=c("CHA","CAR","CAM","EQG","GAB","CNG","ZAI","ANG")
west=c("CAP","MAU","SEN","GAM","GUB","WES","GIN","SIL","LIB","CDI","MAL","BUF","GHA","TOG","BEN","NIG","NIR")
east=c("MOZ","COM","STP","ERI","ETH","UGA","KEN","DJI","SOM","RWA","BUR","TAN","MAA","ZIM","ZAM","MAD" )

In [ ]:
North = ["MAR","DZA","TUN","LBY","EGY","SDN", "SSD"]
South = ["NAM","BWA","ZAF","SWZ","LSO"] 
Central = ["TCD","CAF","CMR","GNQ","GAB","COG","COD","AGO", "STP"]#ok

West = ["MRT","SEN","GMB","GIN","SLE","LBR","GHA","TGO","BEN","NGA", "BFA", "CPV", "CIV", "GNB", "MLI", "NER",'ESH']
East = ["BDI", "COM", "DJI", "ERI", "ETH", "KEN", "MDG", "MWI", "MUS", "MOZ", "RWA", "SYC", "SOM", "UGA", "ZMB", "ZWE", "TZA", "-99"]

In [ ]:
pays.head()

In [ ]:
path = gp.datasets.get_path('naturalearth_lowres')
earth = gp.read_file(path)
africa = earth[earth.continent=='Africa'].copy()

In [ ]:
africa.columns

In [ ]:
t = africa['iso_a3'].copy()
reg = ['NULL' for i in range(len(t))]
for i in range(51):
    x = t.iloc[i]
    if x in North:
        reg[i] = 'North'
        #print(reg)
    elif x in East:
        reg[i] = 'East'
    elif x in West:
        reg[i] = 'West'
    elif x in Central:
        reg[i] = 'Central'
    elif x in South:
        reg[i] = 'South'
    else:
        reg[i] = 'NULL'
print(reg)
africa['Region'] = reg
africa = africa.sort_values('Region').copy()

In [ ]:
africa.sample(5)

In [ ]:
cancer = Bladdersomme
cancer

In [ ]:
LC = cancer['2002']
reg = [0 for i in range(51)]
for i in range(51):
    x = africa['Region'].iloc[i]
    #print(x,)
    if x == 'North':
        reg[i] = LC.iloc[0]
        #print(reg)
    elif x == 'West':
        reg[i] = LC.iloc[1]
    elif x == 'Central':
        reg[i] = LC.iloc[2]
    elif x == 'East':
        reg[i] = LC.iloc[3]
    elif x == 'South':
        reg[i] = LC.iloc[4]
print(africa['Region'].iloc[0]), print(reg[0])
africa['Bladder2002'] = reg

LC = cancer['2008']
reg = [0 for i in range(51)]
for i in range(51):
    x = africa['Region'].iloc[i]
    #print(x,)
    if x == 'North':
        reg[i] = LC.iloc[0]
        #print(reg)
    elif x == 'West':
        reg[i] = LC.iloc[1]
    elif x == 'Central':
        reg[i] = LC.iloc[2]
    elif x == 'East':
        reg[i] = LC.iloc[3]
    elif x == 'South':
        reg[i] = LC.iloc[4]
print(africa['Region'].iloc[0]), print(reg[0])
africa['Bladder2008'] = reg

LC = cancer['2018']
reg = [0 for i in range(51)]
for i in range(51):
    x = africa['Region'].iloc[i]
    #print(x,)
    if x == 'North':
        reg[i] = LC.iloc[0]
        #print(reg)
    elif x == 'West':
        reg[i] = LC.iloc[1]
    elif x == 'Central':
        reg[i] = LC.iloc[2]
    elif x == 'East':
        reg[i] = LC.iloc[3]
    elif x == 'South':
        reg[i] = LC.iloc[4]
#print(africa['Region'].iloc[0]), print(reg[0])
africa['Bladder2018'] = reg

In [ ]:
africa.sample(5)

In [ ]:
cmap = cm.get_cmap('YlGn')
#cmap = cm.get_cmap('winter')
#cmap = cm.get_cmap('tab20c')
#cmap = cm.get_cmap('Greens')
#cmap = cm.get_cmap('tab20c')
#cmap = cm.get_cmap('Set2')
#from cartopy import crs as ccrs
# Generate a figure with two axes, one for CartoPy, one for GeoPandas
vmin, vmax =0, 20
cmap = cm.get_cmap('Oranges')

fig, axs = plt.subplots(1,3,figsize=(10, 5), sharex=True, sharey=True)
fig.suptitle('Incidence rate of Bladder cancer per 100 000 inhabitant')
# Make the CartoPy plot
line1 = africa.plot(column="Bladder2002", cmap=cmap, legend=True, linewidth=0.4, ax=axs[0], edgecolor='0.6',
            legend_kwds={'label': "2002", 'orientation': "horizontal"}, vmin=vmin, vmax=vmax)

line2 = africa.plot(column="Bladder2008", cmap=cmap, legend=True, linewidth=0.4, ax=axs[1], edgecolor='0.6',
            legend_kwds={'label': "2008", 'orientation': "horizontal"}, vmin=vmin, vmax=vmax)

line3 = africa.plot(column="Bladder2018", cmap=cmap, legend=True, linewidth=0.4, ax=axs[2], edgecolor='0.6',
            legend_kwds={'label': "2018", 'orientation': "horizontal"}, vmin=vmin, vmax=vmax)
#plt.colorbar(label='log$_{10}$(population)')
axs[0].axis('off')
axs[1].axis('off')
axs[2].axis('off')
plt.savefig('Figures/blad1.png')
plt.show()

In [ ]:
cancer

In [ ]:
cmap = cm.get_cmap('Set2')
#cmap = cm.get_cmap('winter')
#cmap = cm.get_cmap('tab20c')
ax = cancer.plot.bar(rot=0, cmap=cmap)
plt.suptitle('Bladder cancer')
plt.ylabel('Incidence per 100 000 inhabitant')
plt.savefig('Figures/blad2.png')
plt.show()
cancer.transpose().plot.area(cmap=cmap)
plt.suptitle('Bladder cancer')
plt.ylabel('Incidence per 100 000 inhabitant')
plt.xlabel('Years from 2002 to 2018')
plt.savefig('Figures/blad3.png')
plt.show()

In [ ]:
cancer.plot(kind='bar', stacked=True, cmap=cmap)
plt.suptitle('Bladder cancer')
plt.ylabel('Incidence per 100 000 inhabitant')
plt.savefig('Figures/blad4.png')
plt.show()

In [ ]:
#cmap = cm.get_cmap('Set2')
bdm.transpose().plot.bar(rot=0, cmap=cmap)
plt.suptitle('Bladder cancer for men')
plt.ylabel('Incidence per 100 000 inhabitant')
plt.xlabel('Regions of Africa')
plt.savefig('Figures/blad5.png')
plt.show()

In [ ]:
bdw.transpose().plot.bar(rot=0, cmap=cmap)
plt.suptitle('Bladder cancer for women')
plt.ylabel('Incidence per 100 000 inhabitant')
plt.xlabel('Regions of Africa')
plt.savefig('Figures/blad6.png')
plt.show()

In [ ]:
# A python dictionary

data2002 = {"Male":bdm.iloc[0], "Female":bdw.iloc[0], }; 
data2002 = pd.DataFrame(data=data2002, index=index);

data2008 = {"Male":bdm.iloc[1], "Female":bdw.iloc[1], }; 
data2008 = pd.DataFrame(data=data2008, index=index);

#data2012 = {"Male":bdm.iloc[2], "Female":bdw.iloc[2], }; 
#data2012 = pd.DataFrame(data=data2012, index=index);

data2018 = {"Male":bdm.iloc[2], "Female":bdw.iloc[2], }; 
data2018 = pd.DataFrame(data=data2018, index=index);

fig, axs = plt.subplots(1,3,figsize=(10, 5), sharex=True, sharey=True)
fig.suptitle('Incidence rate of Bladder cancer for Male and for Female per 100 000 inhabitant')
    
p1 = data2002.plot.barh(stacked=False, rot=-15, ax=axs[0], title="2002");
p2 = data2008.plot.barh(stacked=False, rot=-15, ax=axs[1], title="2008");
#p2 = data2012.plot.barh(stacked=True, rot=-15, ax=axs[1], title="2012");
p3 = data2018.plot.barh(stacked=False, rot=-15, ax=axs[2], title="2018");

plt.xlabel('Incidence per 100 000 inhabitant')
plt.ylabel('Regions of Africa')
plt.legend()
plt.savefig('Figures/blad7.png')
plt.show(block=True);